<a href="https://colab.research.google.com/github/tamiti-ruet-ece/Multiclass-Bangla-Sentiment-Analysis/blob/main/Python%20Notebooks/MultiLingual_BERT%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [18]:
import tensorflow as tf
import logging
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv1D,
    Dropout,
    Input,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras import regularizers
from transformers import BertTokenizer, TFBertModel
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import numpy 
import keras
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
import re
import random
import torch

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [4]:
max_length = 100
batch_size = 32
test_size = 0.1
num_class = 6

In [26]:
model_name = "sagorsarker/bangla-bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
df=pd.read_excel('/content/drive/MyDrive/thesis/Multiclass bangla sentiment dataset/Preprocessed Multiclass Bangla Sentiment analysis.xlsx')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6404 entries, 0 to 6403
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6404 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6403 non-null   object
 3   Sentiment   6404 non-null   object
 4   Label       6404 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 250.3+ KB


In [8]:
#df=df.drop_duplicates(subset=['Text'],inplace=True)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(df['Text'],df['Label'] , test_size=test_size, random_state=42)

In [30]:
x_train = x_train.astype(str)
#y_train = y_train.astype(str)

x_test = x_test.astype(str)
#y_test = y_test.astype(str)

In [31]:
def bert_encode(data):
    tokens = tokenizer.batch_encode_plus(
        data, max_length=max_length, padding="max_length", truncation=True
    )
    return tf.constant(tokens["input_ids"])
train_encoded = bert_encode(x_train)
test_encoded = bert_encode(x_test)

train_dataset = (
    tf.data.Dataset.from_tensor_slices((train_encoded, y_train))
    .shuffle(100)
    .batch(batch_size)
).cache()
test_dataset = (
    tf.data.Dataset.from_tensor_slices((test_encoded,y_test))
    .shuffle(100)
    .batch(batch_size)
).cache()

In [32]:
def bert_bangla_sentiment_model():
    bert_encoder = TFBertModel.from_pretrained(model_name, output_attentions=True)
    input_word_ids = Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    last_hidden_states = bert_encoder(input_word_ids)[0]
    clf_output = Flatten()(last_hidden_states)
    net = Dense(512, activation="relu")(clf_output)
    net = Dropout(0.3)(net)
    net = Dense(440, activation="relu")(net)
    net = Dropout(0.3)(net)
    output = Dense(num_class, activation="softmax")(net)
    model = Model(inputs=input_word_ids, outputs=output)
    return model

In [33]:
with strategy.scope():
    model = bert_bangla_sentiment_model()
    adam_optimizer = Adam(learning_rate=1e-5)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=adam_optimizer, metrics=["accuracy"]
    )
    model.summary()

Downloading:   0%|          | 0.00/930M [00:00<?, ?B/s]

Some layers from the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at sagorsarker/bangla-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 100)]             0         
                                                                 
 tf_bert_model_4 (TFBertMode  TFBaseModelOutputWithPoo  164396544
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             100, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=((None, 12, None,             
                             100),                               
                              (None, 12, None, 100),       

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=batch_size,
    epochs=5,
    validation_data=test_dataset,
    verbose=1,
)

Epoch 1/5
  3/181 [..............................] - ETA: 1:53:04 - loss: 2.3359 - accuracy: 0.2812

In [ ]:
model.save_weights('weights.h5', overwrite=True)

In [ ]:
import matplotlib.pyplot as plt
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history["val_" + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, "val_" + string])
    plt.show()
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")